In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

In [2]:
%matplotlib inline
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from numpy import sqrt, cos, sin, pi
from numpy.fft import fft, ifft, rfft, irfft
from tqdm import tqdm
import time

print(tf.__version__)

2.3.0


- 1d closure ([Hammett & Perkins 1990 PRL](https://w3.pppl.gov/~hammett/refs/1990/Hammett_90_PRL_Landau_fluid_corrected.pdf)) written in the Fourier space:
$$\tilde{q}_{k}=-n_{0}\chi_{1}\frac{\sqrt{2}v_{t}}{\left|k\right|}ik\tilde{T}_{k}$$
where $\tilde{q}_k$ is the heatflux in the k-space,  $\tilde{T}_k$ is the temperature fluctuation in the k-space, $\chi_{1}=\frac{2}{\sqrt{\pi}}$, $n_0$ is the background density, $v_t=\sqrt{T_0/m}$ is the thermal speed.
- input/feature: $\tilde{T}$ in the real-space
- output/label: $\tilde{q}$ in the real-space computed (after fft/ifft) from the closue using $\tilde{T}$
- reference: [Ma+2020POP](http://arxiv.org/abs/1909.11509)

### Generate data

In [3]:
nsamples_train = 10000
nsamples_test = 100
nsamples = nsamples_train + nsamples_test

In [4]:
# physics parameters that define the input (feature) and output (label)
n0 = 1.
vt = 1.
chi1 = 2. / sqrt(pi)

lx = 2. * pi
nx = 128
kmax = 8

In [5]:
def make_modes(x, kmax, magnitude=1.):
    """Make many sinusoidal modes with random amplitudes and phases.

    Args:
        x: 1d spatial coordinates.
        kmax: k = 1, 2, ..., kmax.

    Returns:
        modes: Superposition of all modes.
    """
    A = np.random.random((kmax)) * magnitude
    phi = np.random.random((kmax)) * pi * 2
    modes = np.zeros_like(x)
    for k in range(1, kmax):
        A_k = A[k]
        phi_k = phi[k]
        modes += A_k * np.cos(k * x + phi_k)
    return modes


def make_training_data():
    x = np.linspace(0, lx, nx)
    k = np.fft.fftfreq(nx, d=lx/nx) * 2. * pi

    T_all = []
    q_all = []

    coeff_q = -n0 * chi1 * sqrt(2) * vt * 1j * np.sign(k)

    for isample in tqdm(range(nsamples)):
        # Temperature flucutations in real space, T(x)
        T = make_modes(x, kmax)
        # Temperature fluctuations in Fourier spacer, T(k)
        Tk = fft(T)
        # heatflux fluctuation in Fourier space, q(k)
        qk = coeff_q * Tk
        # heatflux fluctuation in real space, q(x)
        q = ifft(qk).real

        # append real-space input/output of this sample
        T_all.append(T)
        q_all.append(q)

    # convert to ndarray of shape (nsamples, nx)
    T_all = np.array(T_all)
    q_all = np.array(q_all)

    return T_all, q_all


def normalize(arr):
    # all data have mean=0
    # XXX use the same max
    return arr / abs(arr).max()

In [6]:
T, q = make_training_data()

100%|██████████| 10100/10100 [00:01<00:00, 7629.62it/s]


In [7]:
T = normalize(T)
q = normalize(q)

In [8]:
train_data_2d = T[:nsamples_train, ...]
train_labels = q[:nsamples_train, ...]

test_data_2d = T[nsamples_train:, ...]
test_labels = q[nsamples_train:, ...]


### Reshape the data

In [9]:
# reshape to [nsamples, nx, nchannels=1]
train_data = train_data_2d.reshape([nsamples_train,nx,1])
test_data  = test_data_2d.reshape([nsamples_test,nx,1])
print(train_data.shape)

(10000, 128, 1)


### Create 1d convlutional model

In [10]:
num_filters = 48
D = 0.10

model = keras.Sequential([
    layers.Conv1D(num_filters, kernel_size=32, padding='same', activation='relu', input_shape=([nx,1])),
    #layers.Dropout(D),
    
    layers.Conv1D(num_filters, kernel_size=32, padding='same', activation='relu'),
    #layers.Dropout(D),

    layers.Conv1D(num_filters, kernel_size=32, padding='same', activation='relu'),
    #layers.Dropout(D),

    layers.Conv1D(num_filters, kernel_size=32, padding='same', activation='relu'),
    #layers.Dropout(D),

    layers.Conv1D(1          , kernel_size=1, padding='same', activation='linear')
])

model.compile(loss='mse',
              optimizer=keras.optimizers.Adam(lr=1e-3),
              metrics=['accuracy', 'mae', 'mse'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 128, 48)           1584      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 128, 48)           73776     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 128, 48)           73776     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 128, 48)           73776     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 128, 1)            49        
Total params: 222,961
Trainable params: 222,961
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [11]:
EPOCHS = 500  # How many times we go through the entire dataset
validation_split = 0.05  # fraction of data to be used as live validation


# https://keras.io/guides/writing_your_own_callbacks/
class PrintLogs(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.time_begin = time.time()

    def on_epoch_end(self, epoch, logs):
        if epoch >0 and (epoch+1) % 100 == 0:
            info = ''
            info += 'EPOCH {}'.format(epoch)
            dt = time.time() - self.time_begin
            dt_per_epoch = dt / (epoch + 1)
            info +='; time total {:g}s per-epoch {:g}s:'.format(dt, dt_per_epoch)
            log_epoch = [
                '{} {:.1e}'.format(key, logs[key]) for key in logs.keys()
            ]
            info += '; '.join(log_epoch)
            print(info)


callbacks = [
    PrintLogs(),
]

# https://keras.io/api/models/model_training_apis/#fit-method
history = model.fit(train_data,
                    train_labels,
                    epochs=EPOCHS,
                    validation_split=validation_split,
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/500
297/297 [==============================] - 40s 133ms/step - loss: 0.0021 - accuracy: 0.0000e+00 - mae: 0.0264 - mse: 0.0021 - val_loss: 7.4068e-04 - val_accuracy: 0.0000e+00 - val_mae: 0.0178 - val_mse: 7.4068e-04
Epoch 2/500
192/297 [==================>...........] - ETA: 13s - loss: 5.7987e-04 - accuracy: 0.0000e+00 - mae: 0.0161 - mse: 5.7987e-04

KeyboardInterrupt: 

### Benchmark the model

In [ ]:
# Use the stats stored in the history object.
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_history(history, step=25):
    fig = plt.figure(figsize=(12, 3))
    hist = pd.DataFrame(history.history).iloc[::step]
    hist['epoch'] = history.epoch[::step]

    plt.subplot(131)
    plt.semilogy(hist['epoch'], hist['mae'], label='training')
    plt.semilogy(hist['epoch'], hist['val_mae'], label='test', alpha=0.8)
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error (mae)')

    plt.subplot(132)
    plt.semilogy(hist['epoch'], hist['mse'], label='training')
    plt.semilogy(hist['epoch'], hist['val_mse'], label='test', alpha=0.8)
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error (mse)')

    plt.tight_layout()


plot_history(history)

In [ ]:
# Compute the overall scores of the model
scores = model.evaluate(test_data, test_labels, verbose=1)

In [ ]:
test_predictions = model.predict(test_data)

In [ ]:
# compare truth and prediction for one or more test samples
for itest in range(1):
    plt.figure()
    plt.plot(test_predictions[itest, :], lw=4, ls='--', label='prediction')
    plt.plot(test_labels[itest, :], label='truth')
    plt.legend()
    plt.xlabel('data index number')
    plt.title('Normalized label values)')

In [ ]:
# do the 45-degree plot; close to the diagonal == good
plt.plot((-1, 1), (-1, 1), c='r')
plt.scatter(test_labels, test_predictions, alpha=0.3)
plt.xlabel('truth')
plt.ylabel('predictions')
plt.title('Normalized label values')
plt.axis('equal')

### Save the model

In [ ]:
model_filename = 'closure-1d-model.h5'
model.save(model_filename)

In [ ]:
import h5py

model_file = h5py.File(model_filename, 'r')


def print_obj(name, obj):
    info = ''
    if isinstance(obj, h5py._hl.dataset.Dataset):
        info = str(obj.shape)
    print('{:42s} {:10s} {}'.format(name, info, type(obj)))


model_file.visititems(print_obj)

model_file.close()